In [ ]:
# import pandas as pd
# from datetime import datetime, timedelta

# # load the data sources
# status_df = pd.read_csv('status.csv')
# hours_df = pd.read_csv('business_hours.csv')
# timezone_df = pd.read_csv('timezones.csv')

# # set store_id of interest
# store_id = 'my_store_id'

# # filter data sources for the store_id of interest
# status_store = status_df[status_df['store_id'] == store_id]
# hours_store = hours_df[hours_df['store_id'] == store_id]
# timezone_store = timezone_df[timezone_df['store_id'] == store_id]

# # convert timestamp_utc to datetime object in UTC
# status_store['timestamp_utc'] = pd.to_datetime(status_store['timestamp_utc'])

# # convert start_time_local and end_time_local to datetime object in local timezone
# hours_store['start_time_local'] = pd.to_datetime(hours_store['start_time_local'])
# hours_store['end_time_local'] = pd.to_datetime(hours_store['end_time_local'])

# # set timezone of local time zone
# local_tz = timezone_store['timezone_str'].values[0] if not timezone_store.empty else 'America/Chicago'

# # set start and end time for the last hour in local time zone
# end_time_local = datetime.utcnow().replace(second=0, microsecond=0).astimezone(local_tz)
# start_time_local = end_time_local - timedelta(hours=1)

# # filter status data within business hours
# hours_status_store = status_store.merge(hours_store, on='dayOfWeek', how='inner')
# hours_status_store = hours_status_store[(hours_status_store['timestamp_utc'] >= start_time_local) & (hours_status_store['timestamp_utc'] <= end_time_local)]
# hours_status_store['status'] = hours_status_store['status'].apply(lambda x: 1 if x == 'active' else 0)

# # calculate uptime in minutes
# uptime_last_hour = (hours_status_store['status'].sum() / 60)

# print(f"Uptime last hour for store {store_id}: {uptime_last_hour} minutes")

In [13]:
import pymongo
import pandas as pd
from datetime import datetime, timedelta
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [14]:
db = client["store"]

In [15]:
store_hours_data = db["store_hours_data"]

In [16]:
store_status_history = db['store_status_history']

In [17]:
store_timezone_data = db['store_timezone_data']

In [46]:
store_timezone_data.find_one()

{'_id': ObjectId('641a84196f22a99af4693a46'),
 'store_id': 8139926242460185114,
 'timezone_str': 'Asia/Beirut'}

In [21]:
# Get the current timestamp in UTC
current_time_utc_last_hr = datetime.utcnow() - timedelta(hours=1)

In [25]:
store_hours_data_df = pd.DataFrame(list(store_hours_data.find({"store_id":8139926242460185114})))

In [26]:
store_hours_data_df

_id             store_id  day start_time_local  \
0  641a84186f22a99af4681f58  8139926242460185114    3         06:00:00   
1  641a84186f22a99af4681f59  8139926242460185114    4         06:00:00   
2  641a84186f22a99af4681f5a  8139926242460185114    2         06:00:00   
3  641a84186f22a99af4681f5b  8139926242460185114    1         06:00:00   
4  641a84186f22a99af4681f5c  8139926242460185114    5         06:00:00   
5  641a84186f22a99af4681f5d  8139926242460185114    0         06:00:00   
6  641a84186f22a99af4681f5e  8139926242460185114    6         06:00:00   

  end_time_local  
0       15:00:00  
1       15:00:00  
2       15:00:00  
3       15:00:00  
4       15:00:00  
5       15:00:00  
6       15:00:00

In [27]:
store_status_history_df = pd.DataFrame(list(store_status_history.find({"store_id":8139926242460185114})))

In [28]:
store_status_history_df

_id             store_id  status  \
0    641a84006f22a99af44c36de  8139926242460185114  active   
1    641a84006f22a99af44c402e  8139926242460185114  active   
2    641a84006f22a99af44c4f01  8139926242460185114  active   
3    641a84006f22a99af44c50af  8139926242460185114  active   
4    641a84006f22a99af44cb6e9  8139926242460185114  active   
..                        ...                  ...     ...   
110  641a84086f22a99af4604257  8139926242460185114  active   
111  641a84086f22a99af460c35f  8139926242460185114  active   
112  641a84086f22a99af460f8cc  8139926242460185114  active   
113  641a84086f22a99af4610935  8139926242460185114  active   
114  641a84086f22a99af4610f4e  8139926242460185114  active   

                      timestamp_utc  
0    2023-01-21 10:00:47.541579 UTC  
1    2023-01-24 20:02:48.842888 UTC  
2    2023-01-18 16:11:17.744136 UTC  
3    2023-01-19 05:04:19.855488 UTC  
4    2023-01-20 08:00:49.668742 UTC  
..                              ...  
110   2023-01-19 14:58:20.04185 UTC  
111   2023-01-20 12:00:54.09101 UTC  
112  2023-01-19 09:26:15.265465 UTC  
113  2023-01-20 09:00:49.144325 UTC  
114  2023-01-25 12:09:33.047112 UTC  

[115 rows x 4 columns]

In [29]:
store_status_history_df['timestamp_utc'] = pd.to_datetime(store_status_history_df['timestamp_utc'])

In [30]:
store_status_history_df

_id             store_id  status  \
0    641a84006f22a99af44c36de  8139926242460185114  active   
1    641a84006f22a99af44c402e  8139926242460185114  active   
2    641a84006f22a99af44c4f01  8139926242460185114  active   
3    641a84006f22a99af44c50af  8139926242460185114  active   
4    641a84006f22a99af44cb6e9  8139926242460185114  active   
..                        ...                  ...     ...   
110  641a84086f22a99af4604257  8139926242460185114  active   
111  641a84086f22a99af460c35f  8139926242460185114  active   
112  641a84086f22a99af460f8cc  8139926242460185114  active   
113  641a84086f22a99af4610935  8139926242460185114  active   
114  641a84086f22a99af4610f4e  8139926242460185114  active   

                       timestamp_utc  
0   2023-01-21 10:00:47.541579+00:00  
1   2023-01-24 20:02:48.842888+00:00  
2   2023-01-18 16:11:17.744136+00:00  
3   2023-01-19 05:04:19.855488+00:00  
4   2023-01-20 08:00:49.668742+00:00  
..                               ...  
110 2023-01-19 14:58:20.041850+00:00  
111 2023-01-20 12:00:54.091010+00:00  
112 2023-01-19 09:26:15.265465+00:00  
113 2023-01-20 09:00:49.144325+00:00  
114 2023-01-25 12:09:33.047112+00:00  

[115 rows x 4 columns]

In [31]:
store_hours_data_df['start_time_local'] = pd.to_datetime(store_hours_data_df['start_time_local'])
store_hours_data_df['end_time_local'] = pd.to_datetime(store_hours_data_df['end_time_local'])

In [36]:
import pytz
end_time_local = datetime.utcnow().replace(second=0, microsecond=0).astimezone(pytz.timezone("America/Chicago"))
start_time_local = end_time_local - timedelta(hours=1)

In [40]:
store_hours_data_df

_id             store_id  day    start_time_local  \
0  641a84186f22a99af4681f58  8139926242460185114    3 2023-03-22 06:00:00   
1  641a84186f22a99af4681f59  8139926242460185114    4 2023-03-22 06:00:00   
2  641a84186f22a99af4681f5a  8139926242460185114    2 2023-03-22 06:00:00   
3  641a84186f22a99af4681f5b  8139926242460185114    1 2023-03-22 06:00:00   
4  641a84186f22a99af4681f5c  8139926242460185114    5 2023-03-22 06:00:00   
5  641a84186f22a99af4681f5d  8139926242460185114    0 2023-03-22 06:00:00   
6  641a84186f22a99af4681f5e  8139926242460185114    6 2023-03-22 06:00:00   

       end_time_local  
0 2023-03-22 15:00:00  
1 2023-03-22 15:00:00  
2 2023-03-22 15:00:00  
3 2023-03-22 15:00:00  
4 2023-03-22 15:00:00  
5 2023-03-22 15:00:00  
6 2023-03-22 15:00:00

In [41]:
# filter status data within business hours
hours_status_store = store_status_history_df.merge(store_hours_data_df, on='store_id', how='inner')
hours_status_store = hours_status_store[(hours_status_store['timestamp_utc'] >= start_time_local) & (hours_status_store['timestamp_utc'] <= end_time_local)]
hours_status_store['status'] = hours_status_store['status'].apply(lambda x: 1 if x == 'active' else 0)

# calculate uptime in minutes
uptime_last_hour = (hours_status_store['status'].sum() / 60)

print(f"Uptime last hour for store {store_id}: {uptime_last_hour} minutes")

NameError: name 'store_id' is not defined

In [45]:
# uptime_last_hour

In [47]:
store_id = 8139926242460185114

In [48]:
store_hours_data_df = pd.DataFrame(list(store_hours_data.find({"store_id":store_id})))

In [49]:
store_status_history_df = pd.DataFrame(list(store_status_history.find({"store_id":store_id})))
store_status_history_df['timestamp_utc'] = pd.to_datetime(store_status_history_df['timestamp_utc'])

In [51]:
store_hours_data_df

_id             store_id  day start_time_local  \
0  641a84186f22a99af4681f58  8139926242460185114    3         06:00:00   
1  641a84186f22a99af4681f59  8139926242460185114    4         06:00:00   
2  641a84186f22a99af4681f5a  8139926242460185114    2         06:00:00   
3  641a84186f22a99af4681f5b  8139926242460185114    1         06:00:00   
4  641a84186f22a99af4681f5c  8139926242460185114    5         06:00:00   
5  641a84186f22a99af4681f5d  8139926242460185114    0         06:00:00   
6  641a84186f22a99af4681f5e  8139926242460185114    6         06:00:00   

  end_time_local  
0       15:00:00  
1       15:00:00  
2       15:00:00  
3       15:00:00  
4       15:00:00  
5       15:00:00  
6       15:00:00

In [57]:
store_status_history_df['timestamp_utc'] = pd.to_datetime(store_status_history_df['timestamp_utc'])

In [54]:
# store_hours_data_df.merge(store_status_history_df, on="store_id")

In [58]:
now = datetime.utcnow()
one_hour_ago = now - timedelta(hours=1)

In [60]:
store_status_history_df

_id             store_id  status  \
0    641a84006f22a99af44c36de  8139926242460185114  active   
1    641a84006f22a99af44c402e  8139926242460185114  active   
2    641a84006f22a99af44c4f01  8139926242460185114  active   
3    641a84006f22a99af44c50af  8139926242460185114  active   
4    641a84006f22a99af44cb6e9  8139926242460185114  active   
..                        ...                  ...     ...   
110  641a84086f22a99af4604257  8139926242460185114  active   
111  641a84086f22a99af460c35f  8139926242460185114  active   
112  641a84086f22a99af460f8cc  8139926242460185114  active   
113  641a84086f22a99af4610935  8139926242460185114  active   
114  641a84086f22a99af4610f4e  8139926242460185114  active   

                       timestamp_utc  
0   2023-01-21 10:00:47.541579+00:00  
1   2023-01-24 20:02:48.842888+00:00  
2   2023-01-18 16:11:17.744136+00:00  
3   2023-01-19 05:04:19.855488+00:00  
4   2023-01-20 08:00:49.668742+00:00  
..                               ...  
110 2023-01-19 14:58:20.041850+00:00  
111 2023-01-20 12:00:54.091010+00:00  
112 2023-01-19 09:26:15.265465+00:00  
113 2023-01-20 09:00:49.144325+00:00  
114 2023-01-25 12:09:33.047112+00:00  

[115 rows x 4 columns]

In [61]:
uptime_last_hour = store_status_history_df['timestamp_utc'].loc[one_hour_ago:now]

TypeError: '<' not supported between instances of 'int' and 'datetime.datetime'

In [132]:
from datetime import datetime, timedelta
import pytz

# set the current time as UTC time
current_time_utc = datetime.now(pytz.utc)

# get the start and end time in local time
start_time_local = datetime.strptime('06:00:00', '%H:%M:%S')
end_time_local = datetime.strptime('15:00:00', '%H:%M:%S')

# convert the local time to UTC time
timezone = pytz.timezone('UTC')
start_time_utc = timezone.localize(datetime.combine(datetime.utcnow().date(), start_time_local.time()))
end_time_utc = timezone.localize(datetime.combine(datetime.utcnow().date(), end_time_local.time()))

# get the store status and timestamp
store_id = '8139926242460185114'
store_status = 'active'
timestamp_utc = datetime.strptime('2023-03-22 13:50:47.541579+00:00', '%Y-%m-%d %H:%M:%S.%f%z').astimezone(pytz.utc)
print(timestamp_utc)
print(current_time_utc - timedelta(hours=1))
# check if the store was active in the last hour
if store_status == 'active' and timestamp_utc > current_time_utc - timedelta(hours=1):
    # calculate the uptime in minutes
    uptime_last_hour = int((min(current_time_utc, end_time_utc) - max(timestamp_utc, start_time_utc)).total_seconds() / 60)
    print(f"The store {store_id} was active for {uptime_last_hour} minutes in the last hour.")
else:
    print(f"The store {store_id} was not active in the last hour.")
    uptime_last_hour = int((min(current_time_utc, end_time_utc) - max(timestamp_utc, start_time_utc)).total_seconds() / 60)
    print(uptime_last_hour)

2023-03-22 13:50:47.541579+00:00
2023-03-22 13:33:13.906996+00:00
The store 8139926242460185114 was active for 42 minutes in the last hour.


In [73]:
import pandas as pd

In [75]:
d = pd.read_csv("store_hours_data.csv")

In [76]:
d

store_id  day start_time_local end_time_local
0      1481966498820158979    4         00:00:00       00:10:00
1      1481966498820158979    2         00:00:00       00:10:00
2      1481966498820158979    0         00:00:00       00:10:00
3      1481966498820158979    1         00:00:00       00:10:00
4      1481966498820158979    5         00:00:00       00:10:00
...                    ...  ...              ...            ...
86193  1774428314218854399    4         00:00:00       23:59:59
86194  1774428314218854399    6         00:00:00       23:59:59
86195  1774428314218854399    3         00:00:00       23:59:59
86196  1774428314218854399    2         00:00:00       23:59:59
86197  1774428314218854399    1         00:00:00       23:59:59

[86198 rows x 4 columns]

In [83]:
pd.to_datetime(d['end_time_local']).dt.time

0        00:10:00
1        00:10:00
2        00:10:00
3        00:10:00
4        00:10:00
           ...   
86193    23:59:59
86194    23:59:59
86195    23:59:59
86196    23:59:59
86197    23:59:59
Name: end_time_local, Length: 86198, dtype: object

In [84]:
# def get_uptime_last_hour():
    
    
from datetime import datetime, timedelta
import pytz

# set the current time as UTC time
current_time_utc = datetime.now(pytz.utc)

In [88]:
current_time_utc.weekday()

2

In [128]:

from datetime import datetime, timedelta
import pytz

# set the current time as UTC time
current_time_utc = datetime.now(pytz.utc)
store_hours_data_a = list(store_hours_data.aggregate([
    {
        '$match': {
            'store_id': 8139926242460185114, 
            'day': 2
        }
    },
    {
        '$project': {
            '_id': 0
        }
    }
]))[0]
# get the start and end time in local time
start_time_local = datetime.strptime(store_hours_data_a.get("start_time_local", "00:00:00"), '%H:%M:%S')
end_time_local = datetime.strptime(store_hours_data_a.get("end_time_local", "24:00:00"), '%H:%M:%S')

# convert the local time to UTC time
timezone = pytz.timezone('UTC')
start_time_utc = timezone.localize(datetime.combine(datetime.utcnow().date(), start_time_local.time()))
end_time_utc = timezone.localize(datetime.combine(datetime.utcnow().date(), end_time_local.time()))
store_status_history_a = list(store_status_history.aggregate([  {    '$match': {'store_id': 8139926242460185114, 'status': 'active'}  },
                                                              {    '$sort': {'timestamp_utc': -1}  },  {    '$limit': 1  }]))[0]

from datetime import datetime, timezone

timestamp_str = store_status_history_a.get("timestamp_utc")
timestamp_str = timestamp_str[:-4]  # Remove the last four characters (i.e. ' UTC')
timestamp_dt = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S.%f')
timestamp_utc = timestamp_dt.replace(tzinfo=timezone.utc)

if store_status_history_a.get("status") == 'active' and timestamp_utc > current_time_utc - timedelta(hours=1):
    # calculate the uptime in minutes
    uptime_last_hour = int((min(current_time_utc, end_time_utc) - max(timestamp_utc, start_time_utc)).total_seconds() / 60)
    print(f"The store {store_id} was active for {uptime_last_hour} minutes in the last hour.")
else:
    print(f"The store {store_id} was not active in the last hour.")

The store 8139926242460185114 was not active in the last hour.
507


In [125]:
store_status_history_a

[{'_id': ObjectId('641a84026f22a99af4503671'),
  'store_id': 8139926242460185114,
  'status': 'active',
  'timestamp_utc': '2023-01-25 18:02:47.427258 UTC'}]

In [121]:
current_time_utc

datetime.datetime(2023, 3, 22, 14, 25, 16, 444078, tzinfo=<UTC>)

In [116]:
store_status_history_a.get("timestamp_utc").strip("UTC")

'2023-01-25 18:02:47.427258 '

In [112]:
store_status_history_a

[{'_id': ObjectId('641a84026f22a99af4503671'),
  'store_id': 8139926242460185114,
  'status': 'active',
  'timestamp_utc': '2023-01-25 18:02:47.427258 UTC'}]

In [101]:
start_time_utc

datetime.datetime(2023, 3, 22, 6, 0, tzinfo=<UTC>)

In [99]:
start_time_local

datetime.datetime(1900, 1, 1, 6, 0)

In [97]:
store_hours_data_a[0]

{'store_id': 8139926242460185114,
 'day': 2,
 'start_time_local': '06:00:00',
 'end_time_local': '15:00:00'}

In [90]:
store_hours_data_df[store_hours_data_df['day']==current_time_utc.weekday()]

_id             store_id  day start_time_local  \
2  641a84186f22a99af4681f5a  8139926242460185114    2         06:00:00   

  end_time_local  
2       15:00:00

In [ ]:
# get the start and end time in local time
start_time_local = store_hours_data_df
end_time_local = store_hours_data_df

In [122]:
from datetime import datetime, timezone

timestamp_str = '2023-01-25 18:02:47.427258 UTC'
timestamp_str = timestamp_str[:-4]  # Remove the last four characters (i.e. ' UTC')
timestamp_dt = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S.%f')
timestamp_utc = timestamp_dt.replace(tzinfo=timezone.utc)

print(timestamp_utc)

2023-01-25 18:02:47.427258+00:00


In [129]:
current_time_utc = datetime.now(pytz.utc)

In [131]:
current_time_utc.weekday()

2